# Trénink s destilací nad datasetem SST2 s modelem BiLSTM
V tomto notebooku je trénován model BiLSTM s odemčenou embedding vrstvou nad původním i augmentovaným datasetem SST2, jako učitelský model je využíván finetunued BERT nad stejným datasetem.

Pro původní i augmentovaný dataset je na základě nalezených hyperparametrů ze sešitu hp_search proveden normální trénink a trénink s destilací znalostí. V rámci tréninků je oproti prohledávání hyperparametrů využito EarlyStoppingu pro zamezení přeučení. Navíc jsou získány také výsledky nad testovací částí datasetu a další metriky využívané v práci (velikost modelu a rychlost inference). Rozdílem oproti ostatním trénovacím notebookům je vygenerování souborů pro nahrání na GLUE Benchmark pro získání skóre nad oficiální testovací částí. 

Při destilaci je využíváno předpočítaných logitů ze sešitu precompute_logits. Konfigurace jednotlivých tréninků odpovídá výstup pěti nejlepších běhů z prohledávání hyperparametrů u dané konfigurace. Maximální délka tréninku je nastavena na 20 epoch. EarlyStopping pracuje s trpělivostí tří epoch.

## Import knihoven a základní nastavení

In [56]:
from datasets import concatenate_datasets, load_from_disk
from transformers import BasicTokenizer, Trainer, EarlyStoppingCallback
from torch.utils.data import DataLoader
import kagglehub
import torch
import base
import os
import copy

Načtení embeddingů.

Načtení datasetu a jeho základní předzpracování (tokenizace, vytvoření slovníků všech tokenů, vytvoření indexu pro GloVe embeddingy).

In [57]:
my_glove = kagglehub.dataset_download("thanakomsn/glove6b300dtxt")
print(my_glove)

/home/jovyan/.cache/kagglehub/datasets/thanakomsn/glove6b300dtxt/versions/1


In [58]:
GLOVE_FILE = f"{my_glove}/glove.6B.300d.txt"
DATASET = "sst2"

In [59]:
train_data = load_from_disk(f"~/data/{DATASET}/train-logits")
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits")

all_train_data = load_from_disk(f"~/data/{DATASET}/train-logits-augmented")
test_blank_data = load_from_disk(f"~/data/{DATASET}/test-blank-logits")

all_data = concatenate_datasets([load_from_disk(file) for file in [f"~/data/{DATASET}/eval-logits", f"~/data/{DATASET}/test-logits", f"~/data/{DATASET}/train-logits-augmented", f"~/data/{DATASET}/test-blank-logits"]])
tokenizer = BasicTokenizer(do_lower_case=True)

Ověření dostupnosti GPU.

In [60]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


Tokenizace.

In [61]:
train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), train_data))
eval_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), eval_data))
test_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), test_data))

all_train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_train_data))
test_data_blank_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), test_blank_data))

all_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_data))

Získání všech unikátních tokenů v datasetu.

In [62]:
vocab = base.get_vocab(all_data_tokens)

Přiřazení indexu jednotlivým tokenům.

In [63]:
word_index = dict(zip(vocab, range(len(vocab))))

Získání indexů z GloVe embeddingů.

In [64]:
embeddings_index = base.get_embeddings_indeces(GLOVE_FILE)

Found 400000 word vectors.


Definice velikosti slovníku a velikosti embedding dimenze. 

In [65]:
print(len(vocab))
num_tokens = len(vocab) + 2
embedding_dim = 300

16152


Vytvoření vazby mezi tokeny (jejich indexy) a embeddingy. Část tokenů nebyla nalezena, což ovšem nepředstavuje problém.

In [66]:
embedding_matrix = base.get_embedding_matrix(num_tokens, embedding_dim, word_index, embeddings_index)

Converted 15775 words (377) misses


Přiřazení indexu tokenům v každé části datasetu.

In [67]:
train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),train_data_tokens))
eval_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),eval_data_tokens))
test_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),test_data_tokens))

all_train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),all_train_data_tokens))
test_data_blank_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),test_data_blank_tokens))

Zarovnání délky všech záznamů.

In [68]:
train_padded_data = list(map(lambda x: base.padd(x,60), train_data_index))
eval_padded_data = list(map(lambda x: base.padd(x,60), eval_data_index))
test_padded_data = list(map(lambda x: base.padd(x,60), test_data_index))

all_train_padded_data = list(map(lambda x: base.padd(x,60), all_train_data_index))
test_blank_padded_data = list(map(lambda x: base.padd(x,60), test_data_blank_index))

Přidání ID tokenů do každé části datasetu.

In [69]:
train_data = train_data.add_column("input_ids", train_padded_data)
eval_data = eval_data.add_column("input_ids", eval_padded_data)
test_data = test_data.add_column("input_ids", test_padded_data)

all_train_data = all_train_data.add_column("input_ids", all_train_padded_data)
test_blank_data = test_blank_data.add_column("input_ids", test_blank_padded_data)

Příprava dataloaderů pro finální ověření rychlosti inference. Testování probíhá pouze nad jedním záznamem z trénovací části.

In [70]:
train_data_gpu = copy.deepcopy(train_data)
train_data_gpu.set_format(type="torch", columns=["input_ids"], device="cuda")
gpu_data_loader = DataLoader(train_data_gpu, batch_size=1, shuffle=False)

train_data_cpu = copy.deepcopy(train_data)
train_data_cpu.set_format(type="torch", columns=["input_ids"], device="cpu")
cpu_data_loader = DataLoader(train_data_cpu, batch_size=1, shuffle=False)

## Normální trénink s původním datasetem

Získání modelu s definovanou embedding vrstvou.

In [71]:
model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=2, freeze_embed = False)

In [72]:
print(model)

BiLSTMClassifier(
  (embedding): Embedding(16154, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=2, bias=True)
)


Konfigurace tréninku, zvolené parametry odpovídají pěti nejlepším výstupům z prohledávání hyperparametrů.

In [73]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base-embedd", logging_dir=f"~/logs/{DATASET}/bilstm-base-embedd", lr=.0007,  epochs=20, weight_decay=0.005)

In [74]:
base.reset_seed()

Konfigurace trenéra s trpělivostí 3 epoch. 

In [75]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)


Spuštění tréninku, výstupy nad validační částí datasetu.

In [76]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.331900,0.462165,0.805046,0.818020,0.806875,0.803632
2,0.190800,0.426498,0.837156,0.838499,0.836554,0.836777
3,0.131600,0.436648,0.844037,0.844576,0.843647,0.843826
4,0.096300,0.566420,0.839450,0.841482,0.838722,0.838962
5,0.071200,0.618627,0.834862,0.840442,0.833670,0.833791
6,0.054400,0.809820,0.834862,0.838584,0.833881,0.834077


TrainOutput(global_step=2526, training_loss=0.14602682275235984, metrics={'train_runtime': 57.6337, 'train_samples_per_second': 18697.055, 'train_steps_per_second': 146.095, 'total_flos': 0.0, 'train_loss': 0.14602682275235984, 'epoch': 6.0})

Přepnutí modelu do evaluačního režimu.


In [77]:
model.eval()

BiLSTMClassifier(
  (embedding): Embedding(16154, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=2, bias=True)
)


Otestování modelu nad umělou testovací částí datasetu.

In [78]:
trainer.evaluate(test_data)

{'eval_loss': 0.18846265971660614,
 'eval_accuracy': 0.9313288789903489,
 'eval_precision': 0.9295388996470819,
 'eval_recall': 0.93211226216994,
 'eval_f1': 0.9306205880720289,
 'eval_runtime': 4.0816,
 'eval_samples_per_second': 3300.167,
 'eval_steps_per_second': 25.97,
 'epoch': 6.0}

Uložení modelu.


In [79]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-base-embedd.pth")

Vygenerování predikcí nad oficiální testovací částí a jejich export pro nahrání na GLUE Benchmark.

In [80]:
test_blank_data.set_format(type="torch", columns=["input_ids"], device="cuda")
test_blank_dataloader = DataLoader(test_blank_data, batch_size=128, shuffle=False)
test_blank_logits = base.generate_logits(test_blank_dataloader, model)

Generating logits for given dataset:   0%|          | 0/15 [00:00<?, ?it/s]

In [81]:
base.generate_real_test_file_sst2(test_blank_logits, f"{os.path.expanduser('~')}/data/{DATASET}/bilstm-base-embedd-test.tsv")

Created output file named: /home/jovyan/data/sst2/bilstm-base-embedd-test.tsv upload it to GLUE benchmark to obtain results!


## Trénink s destilací s původním datasetem

Získání studentského modelu s definovanou embedding vrstvou.

In [82]:
student_model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=2, freeze_embed=False)

Konfigurace tréninku s destilací, zvolené parametry odpovídají pěti nejlepším výstupům z prohledávání hyperparametrů.

In [83]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill-embedd", remove_unused_columns=False, logging_dir=f"~/logs/{DATASET}/bilstm-distill-embedd", lr=.001,  epochs=20, lambda_param=.1, temp=4.5, warmup_steps=15, weight_decay=0.01)

In [84]:
base.reset_seed()

Konfigurace destilačního trenéra s trpělivostí 3 epoch. 

In [85]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

Spuštění tréninku s destilací, výstupy nad validační částí datasetu.

In [86]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.549400,0.871939,0.802752,0.825805,0.805170,0.800016
2,0.270600,0.577139,0.854358,0.854539,0.854119,0.854238
3,0.169600,0.596487,0.849771,0.849713,0.849741,0.849726
4,0.116200,0.737707,0.855505,0.856753,0.854951,0.855200
5,0.084200,0.824428,0.854358,0.856925,0.853572,0.853858
6,0.065000,0.765630,0.847477,0.848587,0.846942,0.847171
7,0.050200,0.777408,0.845183,0.845263,0.845363,0.845178


TrainOutput(global_step=2947, training_loss=0.18646101017988534, metrics={'train_runtime': 67.4533, 'train_samples_per_second': 15975.19, 'train_steps_per_second': 124.827, 'total_flos': 0.0, 'train_loss': 0.18646101017988534, 'epoch': 7.0})

Přepnutí studenta do evaluačního režimu.

In [87]:
student_model.eval()

BiLSTMClassifier(
  (embedding): Embedding(16154, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=2, bias=True)
)


Otestování modelu nad umělou testovací částí datasetu.

In [88]:
trainer.evaluate(test_data)

{'eval_loss': 0.27459990978240967,
 'eval_accuracy': 0.94090571640683,
 'eval_precision': 0.9397652803805101,
 'eval_recall': 0.940591799717313,
 'eval_f1': 0.9401616975573656,
 'eval_runtime': 4.1632,
 'eval_samples_per_second': 3235.493,
 'eval_steps_per_second': 25.461,
 'epoch': 7.0}

Uložení studentského modelu.

In [89]:
torch.save(student_model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-distill-embedd.pth")

Vygenerování predikcí nad oficiální testovací částí a jejich export pro nahrání na GLUE Benchmark.

In [90]:
test_blank_logits = base.generate_logits(test_blank_dataloader, student_model)
base.generate_real_test_file_sst2(test_blank_logits, f"{os.path.expanduser('~')}/data/{DATASET}/bilstm-distill-embedd-test.tsv")

Generating logits for given dataset:   0%|          | 0/15 [00:00<?, ?it/s]

Created output file named: /home/jovyan/data/sst2/bilstm-distill-embedd-test.tsv upload it to GLUE benchmark to obtain results!


## Normální trénink s augmentovaným datasetem

Získání modelu s definovanou embedding vrstvou.

In [91]:
model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=2, freeze_embed=False)

Konfigurace tréninku, zvolené parametry odpovídají pěti nejlepším výstupům z prohledávání hyperparametrů.


In [135]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base-embedd-aug", logging_dir=f"~/logs/{DATASET}/bilstm-base-embedd-aug", lr=.0015,  epochs=20, weight_decay=0.01, warmup_steps=50)

In [136]:
base.reset_seed()

Konfigurace trenéra s trpělivostí 3 epoch. 

In [137]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)


Spuštění tréninku, výstupy nad validační částí datasetu.

In [138]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.157900,0.467077,0.862385,0.864613,0.861665,0.861967
2,0.082900,0.511430,0.868119,0.868661,0.867759,0.867952
3,0.051800,0.628386,0.850917,0.851621,0.850488,0.850690
4,0.036300,0.735948,0.848624,0.848624,0.848741,0.848611
5,0.026600,0.764725,0.861239,0.861219,0.861339,0.861224


TrainOutput(global_step=11475, training_loss=0.07109256187555317, metrics={'train_runtime': 141.0351, 'train_samples_per_second': 41640.119, 'train_steps_per_second': 325.451, 'total_flos': 0.0, 'train_loss': 0.07109256187555317, 'epoch': 5.0})

Přepnutí modelu do evaluačního režimu.


In [139]:
model.eval()

BiLSTMClassifier(
  (embedding): Embedding(16154, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=2, bias=True)
)


Otestování modelu nad umělou testovací částí datasetu.

In [140]:
trainer.evaluate(test_data)

{'eval_loss': 0.2587522268295288,
 'eval_accuracy': 0.9374164810690423,
 'eval_precision': 0.9359272869984216,
 'eval_recall': 0.9376209629198096,
 'eval_f1': 0.936696021942877,
 'eval_runtime': 4.1072,
 'eval_samples_per_second': 3279.584,
 'eval_steps_per_second': 25.808,
 'epoch': 5.0}

Uložení modelu.


In [98]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-base-embedd-aug.pth")

Vygenerování predikcí nad oficiální testovací částí a jejich export pro nahrání na GLUE Benchmark.

In [99]:
test_blank_logits = base.generate_logits(test_blank_dataloader, model)
base.generate_real_test_file_sst2(test_blank_logits, f"{os.path.expanduser('~')}/data/{DATASET}/bilstm-base-embedd-aug-test.tsv")

Generating logits for given dataset:   0%|          | 0/15 [00:00<?, ?it/s]

Created output file named: /home/jovyan/data/sst2/bilstm-base-embedd-aug-test.tsv upload it to GLUE benchmark to obtain results!


## Trénink s destilací s augmentovaným datasetem

Získání studentského modelu s definovanou embedding vrstvou.

In [100]:
student_model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=2, freeze_embed=False)

Konfigurace tréninku s destilací, zvolené parametry odpovídají pěti nejlepším výstupům z prohledávání hyperparametrů.

In [101]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill-embedd-aug", remove_unused_columns=False, logging_dir=f"~/logs/{DATASET}/bilstm-distill-embedd-aug", lr=.0035,  epochs=20, weight_decay=0.005, warmup_steps=20, lambda_param=.7, temp=5)

In [102]:
base.reset_seed()

Konfigurace destilačního trenéra s trpělivostí 3 epoch. 

In [103]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

Spuštění tréninku s destilací, výstupy nad validační částí datasetu.

In [104]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.537700,1.298938,0.853211,0.853498,0.853498,0.853211
2,0.181200,1.340761,0.856651,0.856891,0.856919,0.856651
3,0.131200,1.220432,0.865826,0.865769,0.865844,0.865796
4,0.102900,1.349002,0.850917,0.851110,0.851162,0.850917
5,0.086800,1.368867,0.861239,0.861582,0.861550,0.861238
6,0.076400,1.316256,0.861239,0.861184,0.861213,0.861197


TrainOutput(global_step=13770, training_loss=0.18604126863057144, metrics={'train_runtime': 191.3412, 'train_samples_per_second': 30692.393, 'train_steps_per_second': 239.886, 'total_flos': 0.0, 'train_loss': 0.18604126863057144, 'epoch': 6.0})

Přepnutí studenta do evaluačního režimu.

In [105]:
student_model.eval()

BiLSTMClassifier(
  (embedding): Embedding(16154, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=2, bias=True)
)

Otestování studenta nad umělou testovací částí datasetu.

In [106]:
trainer.evaluate(test_data)

{'eval_loss': 0.39344123005867004,
 'eval_accuracy': 0.9534521158129176,
 'eval_precision': 0.9521642148959297,
 'eval_recall': 0.953787003011884,
 'eval_f1': 0.9529086869884051,
 'eval_runtime': 4.466,
 'eval_samples_per_second': 3016.089,
 'eval_steps_per_second': 23.735,
 'epoch': 6.0}

Uložení studentského modelu.

In [107]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-distill-embedd-aug.pth")

Vygenerování predikcí nad oficiální testovací částí a jejich export pro nahrání na GLUE Benchmark.

In [108]:
test_blank_logits = base.generate_logits(test_blank_dataloader, student_model)
base.generate_real_test_file_sst2(test_blank_logits, f"{os.path.expanduser('~')}/data/{DATASET}/bilstm-distill-embedd-aug-test.tsv")

Generating logits for given dataset:   0%|          | 0/15 [00:00<?, ?it/s]

Created output file named: /home/jovyan/data/sst2/bilstm-distill-embedd-aug-test.tsv upload it to GLUE benchmark to obtain results!


Získání počtu trénovatelných parametrů v modelu. 

In [ ]:
base.count_parameters(student_model)

model size: 24.918MB.
Total Trainable Params: 6532202.


,Modules,Parameters
0,embedding.weight,4846200
1,lstm.weight_ih_l0,360000
2,lstm.weight_hh_l0,360000
3,lstm.bias_ih_l0,1200
4,lstm.bias_hh_l0,1200
5,lstm.weight_ih_l0_reverse,360000
6,lstm.weight_hh_l0_reverse,360000
7,lstm.bias_ih_l0_reverse,1200
8,lstm.bias_hh_l0_reverse,1200
9,fc1.weight,240000


Změření rychlosti inference při použití CPU, 1000 pokusů s jedním záznamem.

In [ ]:
cpu_benchmark = base.BenchMarkRunner(student_model, cpu_data_loader, "cpu", 1000)
print(cpu_benchmark.run_benchmark())

self.infer_speed_comp()
  3.36 ms
  1 measurement, 1000 runs , 4 threads


Změření rychlosti inference při použití GPU, 1000 pokusů s jedním záznamem.

In [ ]:
gpu_benchmark = base.BenchMarkRunner(student_model, gpu_data_loader, "cuda", 1000)
print(gpu_benchmark.run_benchmark())

self.infer_speed_comp()
  1.73 ms
  1 measurement, 1000 runs , 4 threads
